In [50]:
# Dependencies
import requests
import pandas as pd
from config import api_key

In [51]:
# Save config information.
#https://openweathermap.org/current#zip --> INFO OF THE API HERE
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
#query_url = f"{url}appid={api_key}&units={units}&q="
query_url = f"{url}appid={api_key}&units={units}&zip="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=7cebf4d170764e341bf39cd9b93fd6a1&units=metric&zip=


In [52]:
zip_data = pd.read_csv("Data/zips_output.csv")
zip_data.head()

,us_zip_code,type,primary_city,state,latitude,longitude,county,timezone,area_codes,country,estimated_population,created_at,updated_at
0,501,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631,US,384,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
1,602,STANDARD,Aguada,PR,18.38,-67.18,NaN,0,787,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
2,601,STANDARD,Adjuntas,PR,18.16,-66.72,Adjuntas,America/Puerto_Rico,787939,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
3,544,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
4,603,STANDARD,Aguadilla,PR,18.43,-67.15,Aguadilla,America/Puerto_Rico,787,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z


In [53]:
zip_codes= zip_data["us_zip_code"]
updated_zip_codes = []
for zip_code in zip_codes:
    if len(str(zip_code)) == 3:
        updated_zip_codes.append(f'00{zip_code}')
    elif len(str(zip_code)) == 4:
        updated_zip_codes.append(f'0{zip_code}')
    else:
        updated_zip_codes.append(zip_code)

In [54]:

# set up lists to hold reponse info
zip_codes_final = []
lat = []
lng = []
temp = []
pressure = []
humidity = []
wind_speed = []
clouds = []
# Loop through the list of cities and perform a request for data on each
for zip_code in updated_zip_codes:
    try:
        zip_code_varchar = str(zip_code)
        response = requests.get(query_url + zip_code).json()
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        pressure.append(response['main']['pressure'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        clouds.append(response['clouds']['all'])
        zip_codes_final.append(zip_code_varchar)
    except:
        print(f"{zip_code} doesn't exist")


09002 doesn't exist
09001 doesn't exist
09004 doesn't exist
09003 doesn't exist
09006 doesn't exist
09005 doesn't exist
09007 doesn't exist
09010 doesn't exist
09008 doesn't exist
09059 doesn't exist
09013 doesn't exist
09068 doesn't exist
09051 doesn't exist
09049 doesn't exist
09014 doesn't exist
09034 doesn't exist
09028 doesn't exist
09058 doesn't exist
09038 doesn't exist
09009 doesn't exist
09055 doesn't exist
09053 doesn't exist
09012 doesn't exist
09069 doesn't exist
09011 doesn't exist
09054 doesn't exist
09036 doesn't exist
09020 doesn't exist
09079 doesn't exist
09060 doesn't exist
09033 doesn't exist
09063 doesn't exist
09021 doesn't exist
09074 doesn't exist
09046 doesn't exist
09045 doesn't exist
09031 doesn't exist
09042 doesn't exist
09075 doesn't exist
09086 doesn't exist
09056 doesn't exist
09067 doesn't exist
09102 doesn't exist
09095 doesn't exist
09092 doesn't exist
09089 doesn't exist
09094 doesn't exist
09076 doesn't exist
09100 doesn't exist
09096 doesn't exist


In [58]:
# create a data frame from cities, lat, and temp
# create a data frame from cities, lat, and temp
weather_dict = {
    "zip_code": zip_codes_final,
    "lat": lat,
    "lng": lng,
    "temp": temp,
    "pressure": pressure,
    "humidity" : humidity,
    "wind_speed" : wind_speed,
    "clouds" : clouds
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,zip_code,lat,lng,temp,pressure,humidity,wind_speed,clouds
0,00501,40.8154,-73.0451,8.13,1014,86,3.60,100
1,00602,18.3800,-67.1800,23.18,1015,83,1.34,5
2,00601,18.1600,-66.7200,18.66,913,88,5.36,7
3,00544,40.8154,-73.0451,8.13,1014,86,3.60,100
4,00603,18.4300,-67.1500,22.74,1015,83,0.45,5
...,...,...,...,...,...,...,...,...
3234,08903,40.5139,-74.4451,9.30,1013,93,0.45,99
3235,08933,40.4862,-74.4518,9.31,1013,93,0.89,99
3236,08922,40.4800,-74.4500,9.21,1013,93,0.45,99
3237,08988,40.4500,-74.4800,9.16,1014,93,0.45,99


In [57]:
# Output in a csv
weather_data["zip_code"] = weather_data["zip_code"].astype(str)
#weather_data.zip_code= weather_data.zip_code.apply('="{}"'.format)
weather_data.set_index("zip_code",inplace=True)
weather_data.to_csv("cleaned_CSVs/weather_data.csv")